### Load module and data path  

 - module을 import하고 DICOM을 읽기 위한 경로 등을 정의  

In [1]:
import numpy as np
import pydicom
import glob
import os

dcmlists = glob.glob(os.path.join("data/", "*.dcm"))

### Read DICOM data  

 - DICOM 데이터를 읽어드리는 작업을 수행  
 - 이때 의료 영상 데이터가 아닌 경우 예외 처리로 제외  
 - `AttributeError`인 경우 의료 영상 데이터가 아닌 것에 대한 예외 처리  
 - 다른 예외 발생 시 추후에 왜, 어떻게 발생했는지 확인하여 추가 필요  

In [2]:
dcm_list = []

for i in range(len(dcmlists)):
    file_path = dcmlists[i]
    file_name = os.path.basename(file_path)

    try:
        dcm = pydicom.dcmread(file_path)
        dcm_list.append(dcm)
    except AttributeError:
        print("AttributeError : wrong attribute")
    except:
        print("Error")

### DICOM header의 값 중 영상의 raw한 정보와 관련된 부분을 확인  

 - 일반적으로 CT 데이터의 raw한 pixel 값은 0~xxxx등으로 저장되어 있음  
 - 하지만 CT의 경우 물(0)을 기준으로 공기(-1000), 뼈(+400) 등의 CT number를 정의함 ... Hounsfield Unit(HU)  
 - 이를 위해 DICOM header 정보에 `Rescale factor`를 저장하여 사용 ... 이 값을 사용하여 raw를 HU로 변환  
 - `Rescale factor`와 관련된 header 정보는 `Rescale Intercept`와 `Rescale Slope`가 있음  

In [4]:
dcm_test = dcm_list[0]

print("Rescale Intercept : " + str(dcm_test.RescaleIntercept))
print("Rescale Slope : "     + str(dcm_test.RescaleSlope))

Rescale Intercept : -1024
Rescale Slope : 1


 - DICOM 중 의료 영상에 해당하는 `.dcm`의 포맷을 가지는 데이터는 영상으로써 각 pixel마다 intensity를 가짐  
 - 영상 처리에서 이러한 intensity는 어떤 해상도(몇 bits로 표현되는가)를 가지고 표현되는가가 중요함  
 > **Example** 어떤 영상의 intensity가 16 bits로 표현된다고 하자. 이 데이터를 8 bits로 처리되는 변수에 저장하는 경우 intensity의 range에 손상이 발생하게 되고, 정확한 영상처리가 아니게되기 때문.  
 - 위와 같은 이유로 이 의료 영상이 어떤 해상도로 표현되는지를 알아야 함  
 - 이를 알 수 있게 DICOM header에서 tag 값을 지원하며 이름은 `BitsAllocated`임  
 > **Note** 아래의 reference에 의하면 대부분의 DICOM image는 16 bits 또는 8 bits라고 함  
 
 > **Reference** http://dicomiseasy.blogspot.com/2012/08/chapter-12-pixel-data.html  

In [5]:
print("Bits Allocated : "     + str(dcm_test.BitsAllocated))

Bits Allocated : 16


### 이전보다 빠른 방법으로 측정 대역의 intensity를 조절하는 방법  

 - 여기서 이전 방법은 `003_...`에서 설명한 방법을 의미  
 - 이전에는 하나하나 pixel 값을 바꾸는 방식을 사용  
 - 그 이유로는 데이터 해상도의 손실이 우려되었기 때문  
 - 여기서는 위에서 설명한 raw 데이터를 손실없이 받는 방법을 알기에 이를 응용  
 - numpy의 데이터 타입 지원을 사용하여 `BitsAllocated`의 해상도를 지원하는 타입을 선택  
 - 이후 numpy의 코드를 사용하여 한번에 pixel 값을 변경  
 - 여기서 pixel 데이터를 한번에 변경할 때 주의할 점은 2D 데이터를 1D로 만들어서 변경한다는 점  
 > **Note** 2D가 3 by 3 데이터이면 이를 가장 윗줄부터 차례로 기차처럼 연결하여 1D로 만들게 되면 1 by 9가 된다고 생각할 수 있음  

In [7]:
for dcm in dcm_list:
    image = dcm.pixel_array                                           # pixel data 값을 저장
    image = image.astype(np.int16)                                    # BitsAllocated가 16이므로 int16으로 저장
    image[(image > 380) & (image < 1500)] += 1024                     # 특정 intensity 대역을 선택하여 값을 조정

    dcm.pixel_array.flat = np.array(image, dtype=np.int16).flat       # 조정한 데이터 값들을 flat를 사용하여 2D to 1D로 바꾸고 덮어쓰기
    dcm.PixelData = dcm.pixel_array.tobytes()                         # tobytes를 사용하여 type을 맞추어 데이터에 저장해주기
        
    result_file_name = os.path.join("data_cvt/", os.path.splitext(file_name)[0] + str(i) + ".dcm")
    
    dcm.save_as(result_file_name)

    print(result_file_name)

data_cvt/I20101142.dcm
data_cvt/I20101142.dcm
data_cvt/I20101142.dcm
